In [2]:
# Some useful packages
import numpy as np         # linear algebra
import sklearn as sk       # machine learning
import pandas as pd        # reading in data files, data cleaning
import matplotlib.pyplot as plt   # for plotting
import seaborn as sns      # visualization tool
import tensorflow as tf
import keras

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout  

In [ ]:
# Random Forest
# ARIMA
# LSTM

In [ ]:
#load data
df = pd.read_csv('train.csv')

# Random Forest

# LSTM

In [ ]:
#adjust
# model = 
#Rakshit's model
#def build_sequence_model(vocab_size, n_classes):
""" Build a sequence learning model for multi-label classification """
    #define vocab size and n classes
    
inp = kl.Input((None,))
x = kl.LSTM(128, return_sequences=True)(x)
x = kl.GlobalMaxPool1D()(x)
x = kl.Dense(64, activation='sigmoid')(x)
out = kl.Dense(n_classes, activation='sigmoid')(x)
    
model = keras.Model(inputs=inp, outputs=out)
model.summary()


In [ ]:
# sequencial versions
model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#Version 3
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
#    model.add(LSTM(70))
#    model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


# GRU

In [ ]:
inp = kl.Input((None,))
x = kl.Embedding(vocab_size, 64)(inp)
x = kl.GRU(128, return_sequences=True)(x)
x = kl.GlobalMaxPool1D()(x)
x = kl.Dense(64, activation='sigmoid')(x)
out = kl.Dense(n_classes, activation='sigmoid')(x)
    
model = keras.Model(inputs=inp, outputs=out)
model.summary()

# Evaluation